In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

from sklearn.metrics import f1_score, accuracy_score, roc_auc_score

In [17]:
df = pd.read_csv("../Data/volley.csv")

In [18]:
df = df.drop(["away_team","home_team"], axis=1)

In [19]:
df.head()

,away_last_yr_pos,away_prev_game_perf,away_team_av_points,away_team_av_points_conceded,away_team_away_form,away_team_form,away_team_rest_time,away_win_percentage,h2h_form,home_current_pos,...,home_team_av_points_conceded,home_team_form,home_team_home_form,home_team_rest_time,home_win_percentage,match_importance,result,Date,month,year
12,12,0,20.0,20.00,0.0,0.0,7,50.0,0.0,2,...,20.00,0.0,0.0,7,50.0,0,1,7,11,2010
7,6,0,20.0,20.00,0.0,0.0,7,50.0,0.0,8,...,20.00,0.0,0.0,7,50.0,0,1,9,11,2010
13,3,0,20.0,20.00,0.0,0.0,7,50.0,0.0,11,...,20.00,0.0,0.0,7,50.0,0,1,11,11,2010
15,8,0,20.0,20.00,0.0,0.0,7,50.0,0.0,11,...,20.00,0.0,0.0,7,50.0,0,1,11,11,2010
13,3,0,20.4,20.64,-0.2,-0.2,2,0.0,0.0,3,...,20.55,0.4,0.4,2,100.0,0,0,13,11,2010


In [20]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1290 entries, 12 to 2
Data columns (total 23 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   away_last_yr_pos              1290 non-null   int64  
 1   away_prev_game_perf           1290 non-null   int64  
 2   away_team_av_points           1290 non-null   float64
 3   away_team_av_points_conceded  1290 non-null   float64
 4   away_team_away_form           1290 non-null   float64
 5   away_team_form                1290 non-null   float64
 6   away_team_rest_time           1290 non-null   int64  
 7   away_win_percentage           1290 non-null   float64
 8   h2h_form                      1290 non-null   float64
 9   home_current_pos              1290 non-null   int64  
 10  home_last_yr_pos              1290 non-null   int64  
 11  home_prev_game_perf           1290 non-null   int64  
 12  home_team_av_points           1290 non-null   float64
 13  home_team_

In [21]:
y = df["result"]
X = df.drop(["result"], axis=1)

X_train,X_test,y_train,y_test = train_test_split(X, y, test_size=0.2, random_state = 121)

In [22]:
scaler = preprocessing.StandardScaler()
scaler.fit(X_train)
scaled_X_train = scaler.transform(X_train)
scaled_X_test = scaler.transform(X_test)


scaled_X_train = pd.DataFrame(data=scaled_X_train, columns=X_train.columns)
scaled_X_test = pd.DataFrame(data=scaled_X_test, columns=X_test.columns)

# decision trees


In [23]:
# # Define the decision tree classifier

# dt_classifier = DecisionTreeClassifier()

# # Define the grid search parameters
# param_grid = {
#     'criterion': ['gini', 'entropy'],
#     'splitter': ['best', 'random'],
#     'max_depth': [None, 5, 10, 15, 20],
#     'min_samples_split': [2, 5, 10, 15],
#     'min_samples_leaf': [1, 2, 4, 8],
#     'max_features': ['auto', 'sqrt', 'log2', None],
#     'class_weight': [None, 'balanced'],
#     'min_impurity_decrease': [0.0, 0.1, 0.2, 0.3]
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=dt_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(scaled_X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

best_params =  {'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'splitter': 'random'}


In [24]:
# Train the model with the best parameters
best_dt_classifier = DecisionTreeClassifier(**best_params)
best_dt_classifier.fit(scaled_X_train, y_train)


y_pred = best_dt_classifier.predict(scaled_X_test)


test_accuracy = accuracy_score(y_pred,y_test)
test_f1 = f1_score(y_pred, y_test)
test_auc = roc_auc_score(y_pred,y_test)

print("Best Parameters:", best_params)
# print("Best Score:", best_score)
print("Test Accuracy:", test_accuracy)
print("Test f1 score:", test_f1)
print("Test roc auc :", test_auc)

Best Parameters: {'class_weight': None, 'criterion': 'entropy', 'max_depth': 10, 'max_features': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 2, 'min_samples_split': 5, 'splitter': 'random'}
Test Accuracy: 0.6434108527131783
Test f1 score: 0.6642335766423357
Test roc auc : 0.6419715936446799


# k nearest neighbors


In [25]:
# Define the K-Nearest Neighbors classifier

# knn_classifier = KNeighborsClassifier()

# # Define the grid search parameters
# param_grid = {
#     'n_neighbors': [3, 5, 7, 9, 11],  # Number of neighbors to consider
#     'weights': ['uniform', 'distance'],  # Weight function used in prediction
#     'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],  # Algorithm used to compute the nearest neighbors
#     'leaf_size': [10, 20, 30, 40],  # Leaf size passed to BallTree or KDTree
#     # 'p': [1, 2, 3],  # Power parameter for the Minkowski metric
#     # 'metric': ['euclidean', 'manhattan', 'chebyshev']  # Distance metric to use for the tree
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=knn_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(scaled_X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

best_params = {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 11, 'weights': 'uniform'}

In [26]:
# Train the model with the best parameters
best_knn_classifier = KNeighborsClassifier(**best_params)
best_knn_classifier.fit(scaled_X_train, y_train)



y_pred = best_knn_classifier.predict(scaled_X_test)


test_accuracy = accuracy_score(y_pred,y_test)
test_f1 = f1_score(y_pred, y_test)
test_auc = roc_auc_score(y_pred,y_test)

print("Best Parameters:", best_params)
# print("Best Score:", best_score)
print("Test Accuracy:", test_accuracy)
print("Test f1 score:", test_f1)
print("Test roc auc :", test_auc)

Best Parameters: {'algorithm': 'auto', 'leaf_size': 10, 'n_neighbors': 11, 'weights': 'uniform'}
Test Accuracy: 0.7054263565891473
Test f1 score: 0.7164179104477612
Test roc auc : 0.7057692307692307


# Naive Bayes 


In [27]:
# # Define the Gaussian Naive Bayes classifier

# nb_classifier = GaussianNB()

# # Define the grid search parameters
# param_grid = {
#     'var_smoothing': [1e-9, 1e-8, 1e-7, 1e-6, 1e-5],
#     'priors': [None, [0.5, 0.5], [0.3, 0.7], [0.7, 0.3]]  # Prior probabilities for each class
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=nb_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(scaled_X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

best_params = {'priors': None, 'var_smoothing': 1e-09}

In [28]:
# Train the model with the best parameters
best_nb_classifier = GaussianNB(**best_params)
best_nb_classifier.fit(scaled_X_train, y_train)

y_pred = best_nb_classifier.predict(scaled_X_test)


test_accuracy = accuracy_score(y_pred,y_test)
test_f1 = f1_score(y_pred, y_test)
test_auc = roc_auc_score(y_pred,y_test)

print("Best Parameters:", best_params)
# print("Best Score:", best_score)
print("Test Accuracy:", test_accuracy)
print("Test f1 score:", test_f1)
print("Test roc auc :", test_auc)

Best Parameters: {'priors': None, 'var_smoothing': 1e-09}
Test Accuracy: 0.689922480620155
Test f1 score: 0.6491228070175439
Test roc auc : 0.7263368983957219


# support vector machines

In [29]:
# # Define the Support Vector Machine classifier

# svm_classifier = SVC()

# # Define the grid search parameters
# param_grid = {
#     'C': [0.1, 1, 10, 100],
#     'kernel': ['linear', 'poly', 'rbf', 'sigmoid'],
#     # 'degree': [2, 3, 4],  # Only for polynomial kernel
#     # 'gamma': ['scale', 'auto', 0.1, 1, 10]  # 'scale' and 'auto' are default, adding more values for gamma
# }

# # Perform grid search with cross-validation
# grid_search = GridSearchCV(estimator=svm_classifier, param_grid=param_grid, cv=5)
# grid_search.fit(scaled_X_train, y_train)

# # Get the best parameters and best score
# best_params = grid_search.best_params_
# best_score = grid_search.best_score_

best_params = {'C': 10, 'kernel': 'poly'}

In [30]:
# Train the model with the best parameters






best_svm_classifier = SVC(**best_params)
best_svm_classifier.fit(scaled_X_train, y_train)

# Evaluate the model on the test set

y_pred = best_svm_classifier.predict(scaled_X_test)


test_accuracy = accuracy_score(y_pred,y_test)
test_f1 = f1_score(y_pred, y_test)
test_auc = roc_auc_score(y_pred,y_test)

print("Best Parameters:", best_params)
# print("Best Score:", best_score)
print("Test Accuracy:", test_accuracy)
print("Test f1 score:", test_f1)
print("Test roc auc :", test_auc)

Best Parameters: {'C': 10, 'kernel': 'poly'}
Test Accuracy: 0.7170542635658915
Test f1 score: 0.7456445993031359
Test roc auc : 0.7152969295826439
